In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from tensorflow import keras
from tensorflow.keras import layers
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout, LSTM,Input,Activation,Bidirectional,GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from sklearn.metrics import f1_score
import feature_builder
import util
import hyperparameter_tuning
embeddings = None
from hyperopt import hp,STATUS_OK

In [35]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
Y = train_df['target'].values

In [36]:
def prepare_for_cnn(df):
    processed = feature_builder.process_dataset(df)
    return (processed, processed.to_numpy().reshape(processed.shape[0], 1, processed.shape[1]))

In [37]:
processed, X_processed = prepare_for_cnn(train_df)

Percentage of words covered in the embeddings = 0.4937444933920705


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_processed,Y,test_size = 0.33,random_state = 17)

In [2]:
def RNN():
    model = Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU({{64}}),input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)])
    return model

In [15]:
model = RNN()
model.summary()
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 128)               153216    
_________________________________________________________________
dense_6 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 157,377
Trainable params: 157,377
Non-trainable params: 0
_________________________________________________________________


In [21]:
history = model.fit(X_train, y_train, epochs=50, shuffle=False, validation_split=0.0)
util.plot_history(history)

Epoch 1/50
160/160 [==============================] - 2s 14ms/step - loss: 0.4012 - accuracy: 0.8227
Epoch 2/50
160/160 [==============================] - 2s 10ms/step - loss: 0.4011 - accuracy: 0.8251
Epoch 3/50
160/160 [==============================] - 2s 11ms/step - loss: 0.3988 - accuracy: 0.8255
Epoch 4/50
160/160 [==============================] - 1s 7ms/step - loss: 0.3977 - accuracy: 0.8265
Epoch 5/50
160/160 [==============================] - 1s 7ms/step - loss: 0.3961 - accuracy: 0.8271
Epoch 6/50
160/160 [==============================] - 2s 11ms/step - loss: 0.3934 - accuracy: 0.8322
Epoch 7/50
160/160 [==============================] - 2s 13ms/step - loss: 0.3894 - accuracy: 0.8286
Epoch 8/50
160/160 [==============================] - 2s 14ms/step - loss: 0.3918 - accuracy: 0.8322
Epoch 9/50
160/160 [==============================] - 1s 6ms/step - loss: 0.3918 - accuracy: 0.8276
Epoch 10/50
160/160 [==============================] - 1s 6ms/step - loss: 0.3873 - accuracy: 

KeyError: 'val_accuracy'

In [22]:
prediction = model.predict_classes(X_test)

In [23]:
accuracy_score(y_test, prediction)

0.8121766812574612

In [24]:
f1_score(y_test, prediction)

0.742639040348964

Hyperopt (bayesian)

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
Y = train_df['target'].values

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
Y = train_df['target'].values
processed = feature_builder.process_dataset(train_df)
X_processed = processed.to_numpy().reshape(processed.shape[0], 1, processed.shape[1])
X_train, X_test, y_train, y_test = train_test_split(X_processed,Y,test_size = 0.33,random_state = 17)

Percentage of words covered in the embeddings = 0.4937444933920705


In [4]:
def gru(params):
    model = Sequential()
    model.add(Bidirectional(GRU(params['units']),input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(params['units'], activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    
    
    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
    
    result = model.fit(X_train, y_train, epochs=50, shuffle=True, validation_split=0.2)
    #get the highest validation accuracy of the training epochs
    validation_acc = np.amax(result.history['val_accuracy']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}


In [5]:
space = {'units': hp.choice('units',[16,32]),
        'units2': hp.choice('units2',[16,32])}

In [6]:
hyperparameter_tuning.bayesian_optimization(gru,space,max_evals=2)

Epoch 1/50                                           
  1/128 [..............................]             
 - ETA: 0s - loss: 0.7480 - accuracy: 0.5312         
                                                    
  9/128 [=>............................]             
 - ETA: 0s - loss: 0.7326 - accuracy: 0.5486         
                                                    
 14/128 [==>...........................]             
 - ETA: 0s - loss: 0.7407 - accuracy: 0.5491         
                                                    
 19/128 [===>..........................]             
 - ETA: 1s - loss: 0.7396 - accuracy: 0.5428         
                                                    
 22/128 [====>.........................]             
 - ETA: 1s - loss: 0.7435 - accuracy: 0.5540         
                                                    
 25/128 [====>.........................]             
 - ETA: 1s - loss: 0.7415 - accuracy: 0.5437         
                                 

 65/128 [==============>...............]             
 - ETA: 0s - loss: 0.6877 - accuracy: 0.5601         
                                                    
 70/128 [===============>..............]             
 - ETA: 0s - loss: 0.6871 - accuracy: 0.5656         
                                                    
 74/128 [================>.............]             
 - ETA: 0s - loss: 0.6864 - accuracy: 0.5697         
                                                    
 80/128 [=================>............]             
 - ETA: 0s - loss: 0.6831 - accuracy: 0.5734         
                                                    
 86/128 [===================>..........]             
 - ETA: 0s - loss: 0.6839 - accuracy: 0.5701         
                                                    
 91/128 [====================>.........]             
 - ETA: 0s - loss: 0.6829 - accuracy: 0.5711         
                                                    
 97/128 [=====================>...

 - ETA: 0s - loss: 0.6628 - accuracy: 0.5685         
                                                    
 65/128 [==============>...............]             
 - ETA: 0s - loss: 0.6635 - accuracy: 0.5668         
                                                    
 71/128 [===============>..............]             
 - ETA: 0s - loss: 0.6610 - accuracy: 0.5722         
                                                    
 75/128 [================>.............]             
 - ETA: 0s - loss: 0.6619 - accuracy: 0.5713         
                                                    
 79/128 [=================>............]             
 - ETA: 0s - loss: 0.6629 - accuracy: 0.5712         
                                                    
 85/128 [==================>...........]             
 - ETA: 0s - loss: 0.6619 - accuracy: 0.5699         
                                                    
 89/128 [===================>..........]             
 - ETA: 0s - loss: 0.6617 - accura

                                                    
 64/128 [==============>...............]             
 - ETA: 0s - loss: 0.6380 - accuracy: 0.5820         
                                                    
 71/128 [===============>..............]             
 - ETA: 0s - loss: 0.6417 - accuracy: 0.5770         
                                                    
 74/128 [================>.............]             
 - ETA: 0s - loss: 0.6432 - accuracy: 0.5756         
                                                    
 81/128 [=================>............]             
 - ETA: 0s - loss: 0.6416 - accuracy: 0.5810         
                                                    
 88/128 [===================>..........]             
 - ETA: 0s - loss: 0.6416 - accuracy: 0.5792         
                                                    
 93/128 [====================>.........]             
 - ETA: 0s - loss: 0.6427 - accuracy: 0.5773         
                                  

 37/128 [=======>......................]             
 - ETA: 0s - loss: 0.6364 - accuracy: 0.5980         
                                                    
 42/128 [========>.....................]             
 - ETA: 0s - loss: 0.6356 - accuracy: 0.5930         
                                                    
 47/128 [==========>...................]             
 - ETA: 0s - loss: 0.6352 - accuracy: 0.5871         
                                                    
 53/128 [===========>..................]             
 - ETA: 0s - loss: 0.6368 - accuracy: 0.5831         
                                                    
 59/128 [============>.................]             
 - ETA: 0s - loss: 0.6346 - accuracy: 0.5863         
                                                    
 65/128 [==============>...............]             
 - ETA: 0s - loss: 0.6375 - accuracy: 0.5817         
                                                    
 74/128 [================>........

...

                                                                               
 70/128 [===============>..............]                                        
 - ETA: 0s - loss: 0.4456 - accuracy: 0.7902                                    
                                                                               
 73/128 [================>.............]                                        
 - ETA: 0s - loss: 0.4461 - accuracy: 0.7928                                    
                                                                               
 77/128 [=================>............]                                        
 - ETA: 0s - loss: 0.4491 - accuracy: 0.7898                                    
                                                                               
 81/128 [=================>............]                                        
 - ETA: 0s - loss: 0.4512 - accuracy: 0.7894                                    
                                

 - ETA: 0s - loss: 0.4403 - accuracy: 0.7942                                    
                                                                               
 87/128 [===================>..........]                                        
 - ETA: 0s - loss: 0.4382 - accuracy: 0.7945                                    
                                                                               
 90/128 [====================>.........]                                        
 - ETA: 0s - loss: 0.4393 - accuracy: 0.7937                                    
                                                                               
 93/128 [====================>.........]                                        
 - ETA: 0s - loss: 0.4392 - accuracy: 0.7923                                    
                                                                               
 98/128 [=====================>........]                                        
 - ETA: 0s - loss: 0.4381 - accu

 99/128 [======================>.......]                                        
 - ETA: 0s - loss: 0.4502 - accuracy: 0.7825                                    
                                                                               
103/128 [=======================>......]                                        
 - ETA: 0s - loss: 0.4493 - accuracy: 0.7846                                    
                                                                               
106/128 [=======================>......]                                        
 - ETA: 0s - loss: 0.4486 - accuracy: 0.7851                                    
                                                                               
112/128 [=========================>....]                                        
 - ETA: 0s - loss: 0.4460 - accuracy: 0.7871                                    
                                                                               
118/128 [=======================

                                                                               
112/128 [=========================>....]                                        
 - ETA: 0s - loss: 0.4410 - accuracy: 0.7927                                    
                                                                               
116/128 [==========================>...]                                        
 - ETA: 0s - loss: 0.4424 - accuracy: 0.7915                                    
                                                                               
121/128 [===========================>..]                                        
 - ETA: 0s - loss: 0.4393 - accuracy: 0.7931                                    
                                                                               
124/128 [============================>.]                                        
 - ETA: 0s - loss: 0.4431 - accuracy: 0.7906                                    
                                

 - ETA: 0s - loss: 0.4400 - accuracy: 0.7922                                    
                                                                               
125/128 [============================>.]                                        
 - ETA: 0s - loss: 0.4408 - accuracy: 0.7922                                    
                                                                               
128/128 [==============================]                                        
 - ETA: 0s - loss: 0.4412 - accuracy: 0.7914                                    
                                                                               
128/128 [==============================]                                        
 - 2s 13ms/step - loss: 0.4412 - accuracy: 0.7914 - val_loss: 0.4991 - val_accuracy: 0.7539

Best validation acc of epoch:                                                   
0.7539215683937073                                                              
100%|██████████| 2/